In [1]:
# Download VGGish 
# Download the audioset directory using subversion
# !apt-get -qq install subversion
!svn checkout https://github.com/tensorflow/models/trunk/research/audioset

# Download audioset requirements
!pip install numpy scipy
!pip install resampy tensorflow six soundfile

# grab the VGGish model checkpoints & PCA params
!curl -O https://storage.googleapis.com/audioset/vggish_model.ckpt
!curl -O https://storage.googleapis.com/audioset/vggish_pca_params.npz

A    audioset/README.md
A    audioset/mel_features.py
A    audioset/vggish_inference_demo.py
A    audioset/vggish_input.py
A    audioset/vggish_params.py
A    audioset/vggish_postprocess.py
A    audioset/vggish_slim.py
A    audioset/vggish_smoke_test.py
A    audioset/vggish_train_demo.py
Checked out revision 9272.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  277M  100  277M    0     0  12.1M      0  0:00:22  0:00:22 --:--:-- 10.3M  0  12.6M      0  0:00:22  0:00:15  0:00:07 12.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 73020  100 73020    0     0   142k      0 --:--:-- --:--:-- --:--:--  142k


In [3]:
# Test install
!mv audioset/* .
from vggish_smoke_test import *

mv: rename audioset/* to ./*: No such file or directory

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.


Testing your install of VGGish

Log Mel Spectrogram example:  [[-4.47297436 -4.29457354 -4.14940631 ... -3.9747003  -3.94774997
  -3.78687669]
 [-4.48589533 -4.28825497 -4.139964   ... -3.98368686 -3.94976505
  -3.7951698 ]
 [-4.46158065 -4.29329706 -4.14905953 ... -3.96442484 -3.94895483
  -3.78619839]
 ...
 [-4.46152626 -4.29365061 -4.14848608 ... -3.96638113 -3.95057575
  -3.78538167]
 [-4.46152595 -4.2936572  -4.14848104 ... -3.96640507 -3.95059567
  -3.78537143]
 [-4.46152565 -4.29366386 -4.14847603 ... -3.96642906 -3.95061564
  -3.78536116]]
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for upd

In [2]:
import tensorflow as tf
import vggish_slim

vggish_dict = {}
# load the model and get info 
with tf.Graph().as_default(), tf.Session() as sess:
    vggish_slim.define_vggish_slim(training=True)
    vggish_slim.load_vggish_slim_checkpoint(sess,"vggish_model.ckpt")
    
    tvars = tf.trainable_variables()
    tvars_vals = sess.run(tvars)

    for var, val in zip(tvars, tvars_vals):
#         print(var.name, val)  # Prints the name of the variable alongside its value.
        print("%s" % (var.name))
        print("\t" + str(var.shape))
        vggish_dict[var.name] = val
    print("values saved to vggish_dict")


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from vggish_model.ckpt
vggish/conv1/weights:0
	(3, 3, 1, 64)
vggish/conv1/biases:0
	(64,)
vggish/conv2/weights:0
	(3, 3, 64, 128)
vggish/conv2/biases:0
	(128,)
vggish/conv3/conv3_1/weights:0
	(3, 3, 128, 256)
vggish/conv3/conv3_1/biases:0
	(256,)
vggish/conv3/conv3_2/weights:0
	(3, 3, 256, 256)
vggish/conv3/conv3_2/biases:0
	(256,)
vggish/conv4/conv4_1/weights:0
	(3, 3, 256, 512)
vggish/conv4/conv4_1/biases:0
	(512,)
vggish/conv4/conv4_2/weights:0
	(3, 3, 512, 512)
vggish/conv4/conv4_2/biases:0
	(512,)

In [5]:
import torch as tr
import math

class Conv2dSame(tr.nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1):
        super(Conv2dSame, self).__init__()
        self.F = kernel_size
        self.S = stride
        self.D = dilation
        self.layer = tr.nn.Conv2d(in_channels, out_channels, kernel_size, stride, dilation=dilation)

    def forward(self, x_in):
        N, C, H, W = x_in.shape
        H2 = math.ceil(H / self.S)
        W2 = math.ceil(W / self.S)
        Pr = (H2 - 1) * self.S + (self.F - 1) * self.D + 1 - H
        Pc = (W2 - 1) * self.S + (self.F - 1) * self.D + 1 - W
        x_pad = tr.nn.ZeroPad2d((Pr//2, Pr - Pr//2, Pc//2, Pc - Pc//2))(x_in)
        x_out = self.layer(x_pad)
        return x_out

In [11]:
import torch as tr
import torch.nn as nn
import numpy as np
# Define torch model for vggish
# From vggish_slim:
# The VGG stack of alternating convolutions and max-pools.
#     net = slim.conv2d(net, 64, scope='conv1')
#     net = slim.max_pool2d(net, scope='pool1')
#     net = slim.conv2d(net, 128, scope='conv2')
#     net = slim.max_pool2d(net, scope='pool2')
#     net = slim.repeat(net, 2, slim.conv2d, 256, scope='conv3')
#     net = slim.max_pool2d(net, scope='pool3')
#     net = slim.repeat(net, 2, slim.conv2d, 512, scope='conv4')
#     net = slim.max_pool2d(net, scope='pool4')
#     # Flatten before entering fully-connected layers
#     net = slim.flatten(net)
#     net = slim.repeat(net, 2, slim.fully_connected, 4096, scope='fc1')
#     # The embedding layer.
#     net = slim.fully_connected(net, params.EMBEDDING_SIZE, scope='fc2')


vggish_list = list(vggish_dict.values())
def param_generator():
    param = vggish_list.pop(0)
    transposed = np.transpose(param)
    to_torch = tr.from_numpy(transposed)
    result = tr.nn.Parameter(to_torch)
    yield result

class VGGish(nn.Module):
    
    def __init__(self):
        super(VGGish, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.MaxPool2d(2, 2))
        self.embeddings = nn.Sequential(
            nn.Linear(512*7*5, 4096),
            nn.Linear(4096, 4096),
            nn.Linear(4096, 128))
        
        # extract weights from `vggish_list`
        for seq in (self.features, self.embeddings):
            for layer in seq:
                if type(layer).__name__ != "MaxPool2d":
                    layer.weight = next(param_generator())
                    layer.bias = next(param_generator())
            
    def forward(self, x):
        x = self.features(x)
        x = x.view(-1)
        x = self.embeddings(x)
        return x

net = VGGish()
net.eval()

# Test to see if the weights have loaded smoothly 
data = tr.rand(64,96)[None,None,:,:]
print("If it's [128] we're good!")
print(net(data).shape)

# Save to disk
tr.save(net.state_dict(), "./vggish.pth")

If it's [128] we're good!
torch.Size([128])
